In [14]:
import robustbench
from tqdm import tqdm
import os
import torch
from autoattack import AutoAttack
from robustbench.data import _load_dataset
from torchvision import transforms, datasets
import robustbench
from robustbench.utils import load_model
import multiprocessing as mp
import pandas as pd
import sys
sys.path.append("./retrain_robustbench")
import data
import models
from glob import glob
from collections import defaultdict
import multiprocessing as mp
import numpy as np
import onnx
import struct
from joblib import Parallel, delayed
from glob import glob
import torchvision

In [25]:
gpus = [0, 1, 2, 3]
workers_per_gpu = 1

def gpu_worker(gpu, q):
    
    DEVICE = f"cuda:{gpu}"
    
    testset = torchvision.datasets.CIFAR10(root='/data/datasets/cifar10', train=False, download=True, transform=transforms.ToTensor())
    x_test, y_test = _load_dataset(testset)
    x_test, y_test = x_test.to(DEVICE), y_test.to(DEVICE)
    
    while not q.empty():
        file = q.get()
        model_name = os.path.abspath(os.path.join(file, '../../..'))
        model_name = model_name[model_name.rfind("/")+1:]
        model = models.get_model(model_name, dataset="cifar10")
        state = torch.load(file, map_location=DEVICE)
        model.load_state_dict(dict((key[len("model."):], value) for (key, value) in state["state_dict"].items()))        
        model.to(DEVICE)
        model.eval()
        adversary = AutoAttack(model, norm='Linf', eps=8/255, log_path=f"robust_benchmark/cifar10/{model_name}_cifar10.log", device=DEVICE)
        adversary.run_standard_evaluation(x_test, y_test)
        
if __name__ == '__main__':
    que = mp.Queue()
    
    for file in glob("/ssd1/checkpoints_robustbench_models_normal_training/cifar10/*/*/checkpoints/*.ckpt"):
        model_name = os.path.abspath(os.path.join(file, '../../..'))
        model_name = model_name[model_name.rfind("/")+1:]
        logpath = f"robust_benchmark/cifar10/{model_name}_cifar10.log"
        if not os.path.exists(logpath):
            print(">>>", file)
            que.put(file)
    
    processes = []
    for gpu in gpus * workers_per_gpu:
        p = mp.Process(target=gpu_worker, args=(gpu, que))
        p.start()
        processes.append(p)

>>> /ssd1/checkpoints_robustbench_models_normal_training/cifar10/robustbench_Rebuffi2021Fixing_106_16_cutmix_ddpm/version_5/checkpoints/epoch=112-step=44069.ckpt
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square


Process Process-41:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_96557/630013971.py", line 22, in gpu_worker
    adversary.run_standard_evaluation(x_test, y_test)
  File "/opt/conda/lib/python3.8/site-packages/autoattack/autoattack.py", line 88, in run_standard_evaluation
    checks.check_dynamic(self.model, x_orig[:bs].to(self.device), self.is_tf_model,
  File "/opt/conda/lib/python3.8/site-packages/autoattack/checks.py", line 101, in check_dynamic
    model(x)
  File "/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/robustbench/model_zoo/architectures/dm_wide_resnet.py", line 190, in forward
    out = self.layer(out)
 

In [22]:
gpus = [0, 1, 2, 3]
workers_per_gpu = 1

def gpu_worker(gpu, q):
    
    DEVICE = f"cuda:{gpu}"
    
    testset = torchvision.datasets.CIFAR100(root='/data/datasets/cifar100', train=False, download=True, transform=transforms.ToTensor())
    x_test, y_test = _load_dataset(testset)
    x_test, y_test = x_test.to(DEVICE), y_test.to(DEVICE)
    
    while not q.empty():
        file = q.get()
        model_name = os.path.abspath(os.path.join(file, '../../..'))
        model_name = model_name[model_name.rfind("/")+1:]
        model = models.get_model(model_name, dataset="cifar100")
        state = torch.load(file, map_location=DEVICE)
        model.load_state_dict(dict((key[len("model."):], value) for (key, value) in state["state_dict"].items()))        
        model.to(DEVICE)
        model.eval()
        adversary = AutoAttack(model, norm='Linf', eps=8/255, log_path=f"robust_benchmark/cifar100/{model_name}_cifar100.log", device=DEVICE)
        adversary.run_standard_evaluation(x_test, y_test)
        
if __name__ == '__main__':
    que = mp.Queue()
    
    for file in glob("/ssd1/checkpoints_robustbench_models_normal_training/cifar100/*/*/checkpoints/*.ckpt"):
        que.put(file)
    
    processes = []
    for gpu in gpus * workers_per_gpu:
        p = mp.Process(target=gpu_worker, args=(gpu, que))
        p.start()
        processes.append(p)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 63.45%
initial accuracy: 79.16%
initial accuracy: 61.46%
initial accuracy: 78.75%
apgd-ce - 1/26 - 250 out of 250 successfully perturbed
apgd-ce - 2/26 - 250 out of 250 successfully perturbed
apgd-ce - 3/26 - 250 out of 250 successfully perturbed
apgd-ce - 4/26 - 250 out of 250 successfully perturbed
apgd-ce - 1/32 - 243 out of 250 successfully perturbed
apgd-ce - 5/26 - 250 out of 250 successfully perturbed
ap